## Task 3

In [3]:
import sys
import re

#read supertags
supertags = {}
for line in open("../resources/supertags.txt"):
    word, tag = line.split()
    supertags[word] = tag

#get unigrams
unigrams = {}
for line in open("../poleval_corpus/polish_corpora.txt"):
    
    line = re.sub('([.,!?()])', r' \1 ', line)
    line = re.sub('\s{2,}', ' ', line)
    
    for w in line.split():
        word = w.lower()
        if word in supertags:
            tag = supertags[word]
        else:
            tag = ('^' + word)[-3:]
        
        if tag not in unigrams:
            unigrams[tag] = {}
        
        if word not in unigrams[tag]:
            unigrams[tag][word] = 0
        
        unigrams[tag][word] += 1




In [4]:
import random


def new_sentence(sentence):
    res = []
    for word in sentence.lower().split():
        if word in supertags:
            tag = supertags[word]
        else:
            tag = ('^' + word)[-3:]
        
        if tag not in unigrams:
            res.append(word)
        else:
            word = random.choices(list(unigrams[tag].keys()), weights=unigrams[tag].values())
        res.append(word[0])
    return ' '.join(res)



In [5]:

print(new_sentence("Siała baba mak"))
print(new_sentence("Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę"))
print(new_sentence("No nie powiem żeby to dobrze działało"))
print(new_sentence("Bez sensu w takim modelu unigramowym rzeczy wychodza"))


broniła kobieta kościół
nawierzchniowy minister przeprowadził w niewielkiej cenie w pokrytą porę
a nie zamknę w to przeciwieństwie potrzebowało
list motywu z wszystkim pożarze ostrosłupowym odpowiedzi przychodza


## Task 4

In [6]:

bigrams = {}


for line in open("../poleval_corpus/poleval_2grams.txt"):
    count, prev, word = line.split()
    count = int(count)

    
    if prev not in bigrams:
        bigrams[prev] = {}
    bigrams[prev][word] = count
    


In [9]:
import random


#dla kazdego czesciowego wyniku zrobie 10 losowan, sprawdze czy jest bigram jak jest to dodam do listy:
#zostawie najlepsze 5

def new_sentence(sentence):
    res = []
    
    prev_list = [(len(bigrams["<BOS>"]), ["<BOS>"])]
    
    for word in sentence.lower().split() + ["<EOS>"]:
        
        new_list = []

        if word in supertags:
            tag = supertags[word]
        else:
            tag = ('^' + word)[-3:]
        
        for cont , s in prev_list:
            prev = s[-1]
            
            if cont == 0:
                continue
            
            
            if tag not in unigrams:
                new_words = [word]
            else:
                new_words = random.choices(list(unigrams[tag].keys()), weights=unigrams[tag].values(), k=25)
            
            #print(new_words)
            
            for new_word in new_words:
                #prev is in bigrams
                if new_word in bigrams[prev]:
                    if new_word in bigrams:
                        new_list.append((len(bigrams[new_word]), s + [new_word]))
                    else:
                        new_list.append((0, s + [new_word]))
            
        if not new_list:
            
            if tag not in unigrams:
                new_words = [word]
            else:
                new_words = random.choices(list(unigrams[tag].keys()), weights=unigrams[tag].values(), k=25)
                
                
            for new_word in new_words:
                if new_word in bigrams:
                    new_list.append((len(bigrams[new_word]), s + ["|", new_word]))
                else:
                    new_list.append((0, s + ["|", new_word]))
        

        prev_list = list(sorted(new_list, reverse=True))[:5]
        
            
            
    return ' '.join(prev_list[0][1])

In [10]:
print(new_sentence("Siała baba mak"))
print(new_sentence("Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę"))
print(new_sentence("No nie powiem żeby to dobrze działało"))
print(new_sentence("Bez sensu w takim modelu unigramowym rzeczy wychodza"))

<BOS> liczyła kilka raz | <EOS>
<BOS> społeczny pan otrzymał i niewielkiej formie i znaną funkcję | <EOS>
<BOS> w nie powiem i to dobrze mogło | <EOS>
<BOS> bez dnia i tym czasie turowym | nieruchomości wchodza | <EOS>
